In [1]:
# !pip install simplejson

In [2]:
from pymongo import MongoClient
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import simplejson as json
import itertools
from functools import cmp_to_key
import networkx as nx

from IPython.display import display, Image, JSON
from ipywidgets import widgets, Image, HBox, VBox, Button, ButtonStyle

from lib.image_dedup import make_hashes, calculate_distance, hashes_diff
from lib.PersistentSet import PersistentSet
from lib.sort_things import post_score, sort_posts, sort_images
from lib.parallel import parallel

In [3]:
images_dir = Path('./images')
handmade_dir = Path('./handmade')
handmade_dir.mkdir(exist_ok=True)

In [4]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
posts = db['posts']
imagemeta = db['imagemeta']
imagededup = db['imagededup']

In [5]:
imagededup.drop()
for i in imagemeta.find():
    imagededup.insert_one(i)

# Load image metadata

In [6]:
imageDedup = [m for m in imagemeta.find()]
imageDedup.sort(key=lambda x: x['image_id'])

In [7]:
phash_to_idx_mapping = {}
for i in range(len(imageDedup)):
    phash = imageDedup[i]['phash']
    l = phash_to_idx_mapping.get(phash, [])
    l.append(i)
    phash_to_idx_mapping[phash] = l
def phash_to_idx (phash):
    return phash_to_idx_mapping.get(phash, None)

In [8]:
image_id_to_idx_mapping = {imageDedup[i]['image_id']:i for i in range(len(imageDedup))}
def image_id_to_idx (image_id):
    return image_id_to_idx_mapping.get(image_id, None)

# Calculate distance

## Hash distance

In [9]:
image_hashes = [make_hashes(m) for m in imageDedup]

In [10]:
distance = calculate_distance(image_hashes)

In [11]:
# distance2 = np.ndarray([len(image_hashes), len(image_hashes)])
# for i in tqdm(range(len(image_hashes))):
#     for j in range(i+1):
#         diff = hashes_diff(image_hashes[i], image_hashes[j])
#         distance2[i, j] = diff
#         distance2[j, i] = diff
# np.array_equal(distance, distance2)

In [12]:
pdistance = calculate_distance(image_hashes, hash_type='phash')

## Find duplicated pairs from distance matrix

In [13]:
def set_distance (hashes, value, mat=distance):
    phash_x = hashes[0]
    phash_y = phash_x if len(hashes) == 1 else hashes[1]
    idx_x = phash_to_idx(phash_x)
    idx_y = phash_to_idx(phash_y)
    if idx_x == None or idx_y == None:
        return
    for s in itertools.product(idx_x, idx_y):
        i, j = s
        mat[i, j] = value
        mat[j, i] = value

def set_distance_pairs (phash_pairs, value, mat=distance):
    for p in phash_pairs:
        set_distance(list(p), value, mat=mat)

In [14]:
auto_duplicated_image_phash_pairs = PersistentSet()
auto_duplicated_image_phash_pairs.set_file(handmade_dir/'auto_duplicated_image_phash_pairs.json')

In [15]:
for i in tqdm(range(distance.shape[0])):
    for j in range(i):
        if distance[i, j] <= 1: # checked, all distance <= 1 are duplicated
            auto_duplicated_image_phash_pairs.add(frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']]))

In [16]:
for i in tqdm(range(pdistance.shape[0])):
    for j in range(i):
        if pdistance[i, j] <= 1: # checked, all distance <= 1 are duplicated
            auto_duplicated_image_phash_pairs.add(frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']]))

In [17]:
auto_duplicated_image_phash_pairs.save()

## Apply information from meta data

In [18]:
duplicated_post_image_phash_pairs = PersistentSet()
duplicated_post_image_phash_pairs.set_file(handmade_dir/'duplicated_post_image_phash_pairs.json')

for p in tqdm(posts.find()):
    if len(p.get('duplicated_posts', [])) == 0:
        continue

    dp_phashes = {i['phash']
                    for dp in p['duplicated_posts']
                    for i in imagemeta.find({'post_id': dp})}
    if len(dp_phashes) > 1:
        print(f"More than 1 dp image {p['post_id']}")
#         print(f"{p['duplicated_posts']} {dp_phashes}")
        continue

    phashes = [i['phash'] for i in imagemeta.find({'post_id': p['post_id']})]
    if len(phashes) > 1:
        print(f"More than 1 image {p['post_id']} {phashes}")
        continue
    for s in itertools.product(dp_phashes, phashes):
        fs = frozenset(s)
        if len(fs) > 1:
            duplicated_post_image_phash_pairs.add(fs)

duplicated_post_image_phash_pairs.save()

More than 1 dp image reddit/coolguides/egra3f
More than 1 dp image reddit/dataisbeautiful/eawwrn
More than 1 dp image reddit/PewdiepieSubmissions/duc30c
More than 1 dp image reddit/programming/d2qrx6
More than 1 image reddit/dataisugly/c9dmi4 ['b89ef6818f27981c', 'ba1eaf0f9284a99c']
More than 1 dp image reddit/dataisbeautiful/c918xs
More than 1 dp image reddit/dataisbeautiful/bx1fzb
More than 1 dp image reddit/dataisugly/bmp6dx
More than 1 dp image reddit/ireland/bpcyrr
More than 1 dp image reddit/sixers/bmoi3z
More than 1 dp image reddit/Netherlands/bpcsix
More than 1 dp image reddit/germany/bpck0u
More than 1 dp image reddit/europe/bpceo9
More than 1 dp image reddit/educationalgifs/bjxrje
More than 1 dp image reddit/worldpolitics/av0d28
More than 1 dp image reddit/Artifact/a5wa9g
More than 1 dp image reddit/singapore/a2327a
More than 1 dp image reddit/dataisbeautiful/9q3l0i
More than 1 dp image reddit/CryptoCurrency/98w53e
More than 1 dp image reddit/totallynotrobots/90nsmq
More than

In [19]:
related_album_image_phash_pairs = PersistentSet()
related_album_image_phash_pairs.set_file(handmade_dir/'related_album_image_phash_pairs.json')

for album in tqdm({i['album'] for i in imagemeta.find({'album': {'$exists': True, '$ne': ''}})}):
    ra_phashes = [i['phash'] for i in imagemeta.find({'album': album})]
    if len(ra_phashes) <= 1:
        print(f"Only 1 or less image {album} {ra_phashes}")

    for s in itertools.product(ra_phashes, ra_phashes):
        fs = frozenset(s)
        if len(fs) > 1:
            related_album_image_phash_pairs.add(fs)

related_album_image_phash_pairs.save()

Only 1 or less image 2ophbe ['8c7233d364cc6673']



## Apply manual labeled data

In [20]:
duplicated_image_phash_pairs = PersistentSet.load_set(handmade_dir/'duplicated_image_phash_pairs.json')
not_duplicated_image_phash_pairs = PersistentSet.load_set(handmade_dir/'not_duplicated_image_phash_pairs.json')
related_image_phash_pairs = PersistentSet.load_set(handmade_dir/'related_image_phash_pairs.json')
invalid_image_phashes = PersistentSet.load_set(handmade_dir/'invalid_image_phashes.json')

In [21]:
duplicated_image_phash_pairs.persist_add(frozenset(['ea11bc7f91b99094', 'c1c417436a1b6d3f']))
duplicated_image_phash_pairs.persist_add(frozenset(['ecc6913b9a68c365', 'c66e3b313886c7ca']))
duplicated_image_phash_pairs.persist_add(frozenset(['fed095305e626a9a', 'd2d4d6d093d8926b']))
duplicated_image_phash_pairs.persist_add(frozenset(['afdfc0402eb59692', 'd9b681e04d3b8e6c']))
duplicated_image_phash_pairs.persist_add(frozenset(['949a13b659757263', 'afdfc0402eb59692']))
duplicated_image_phash_pairs.persist_add(frozenset(['fe81837a94e3807e', 'c1cf027147e6e1e9']))
duplicated_image_phash_pairs.persist_add(frozenset(['d1c66d8132b783b5', 'd445eb81b6aed12a']))
duplicated_image_phash_pairs.persist_add(frozenset(['e79e9068e7c29b84', 'ff9911311f1b1915']))
duplicated_image_phash_pairs.persist_add(frozenset(['ea97856e1e216987', 'b0c9ceb0d2c63c6d']))
duplicated_image_phash_pairs.persist_add(frozenset(['d25264dfa9659392', 'd1c66d8132b783b5']))
duplicated_image_phash_pairs.persist_add(frozenset(['c13e3ae10e70fd86', 'fe81837a94e3807e']))
duplicated_image_phash_pairs.persist_add(frozenset(['b163cb988e8e6d43', 'ead685999b447b60']))
duplicated_image_phash_pairs.persist_add(frozenset(['ead685999b447b60', 'fb65953b889dd108']))
duplicated_image_phash_pairs.persist_add(frozenset(['8278d797282a8b9f', '82897eff698302f4']))
duplicated_image_phash_pairs.persist_add(frozenset(['fed095305e626a9a', 'afdfc0402eb59692']))

In [22]:
related_image_phash_pairs.persist_add(frozenset(['c13e3ae10e70fd86', 'c1cf027147e6e1e9']))
related_image_phash_pairs.persist_add(frozenset(['b63e4c6e2d6c0c2d', 'e66f1b1b1b64254c']))
related_image_phash_pairs.persist_add(frozenset(['aaa3a383a3a4a3b7', 'ff01805b017f057f']))
related_image_phash_pairs.persist_add(frozenset(['9ba4581be45abd46', '93adc404dbf8f981']))
related_image_phash_pairs.persist_add(frozenset(['c6158b9b14647b3e', 'c45485cb9b96e6e1']))
related_image_phash_pairs.persist_add(frozenset(['9fe1a39c414f20bb', '8fd17949388bba86']))
related_image_phash_pairs.persist_add(frozenset(['c0953f6b3fc1c095', 'c014bf62bf1fe144']))
related_image_phash_pairs.persist_add(frozenset(['d2d2ad39c38c3ab1', 'ec929b6d9628c33c']))

In [23]:
set_distance_pairs(auto_duplicated_image_phash_pairs, 0)
set_distance_pairs(duplicated_post_image_phash_pairs, 0)
set_distance_pairs(duplicated_image_phash_pairs, 0)
set_distance_pairs(not_duplicated_image_phash_pairs, 60)
set_distance_pairs(related_album_image_phash_pairs, 60)
set_distance_pairs(related_image_phash_pairs, 60)

related_distance = np.full(distance.shape, 60)
set_distance_pairs(related_album_image_phash_pairs, 0, mat=related_distance)
set_distance_pairs(related_image_phash_pairs, 0, mat=related_distance)

# Human in the Loop

In [24]:
def make_dedup_box (idx_x, idx_y, default=None):
    image_x = imageDedup[idx_x]
    phash_x = image_x['phash']
    image_y = imageDedup[idx_y]
    phash_y = image_y['phash']
    hash_pair = frozenset([phash_x, phash_y])

    yes_btn = widgets.Button(description="Duplicated", button_style='success')
    no_btn = widgets.Button(description="Not", button_style='info')
    related_btn = widgets.Button(description="Related", button_style='warning')
    invalid_x_btn = widgets.Button(description="X Invalid")
    invalid_y_btn = widgets.Button(description="Y Invalid")
    reset_btn = widgets.Button(description="Reset")
    output = widgets.Output()

    def on_yes (btn):
        with output:
            if hash_pair in not_duplicated_image_phash_pairs:
                not_duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Not')
            duplicated_image_phash_pairs.persist_add(hash_pair)
            print('Duplicated')

    def on_no (btn):
        not_duplicated_image_phash_pairs.persist_add(hash_pair)
        with output:
            print('Not')

    def on_related (btn):
        with output:
            if hash_pair in not_duplicated_image_phash_pairs:
                not_duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Not')
            related_image_phash_pairs.persist_add(hash_pair)
            print('Related')

    def on_invalid_x (btn):
        invalid_image_phashes.persist_add(phash_x)
        with output:
            print('Invalid X')

    def on_invalid_y (btn):
        invalid_image_phashes.persist_add(phash_y)
        with output:
            print('Invalid Y')

    def on_reset (btn):
        with output:
            if hash_pair in duplicated_image_phash_pairs:
                duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Duplicated')
            if hash_pair in not_duplicated_image_phash_pairs:
                not_duplicated_image_phash_pairs.persist_remove(hash_pair)
                print('-Not')
            if hash_pair in related_image_phash_pairs:
                related_image_phash_pairs.persist_remove(hash_pair)
                print('-Related')
            if phash_x in invalid_image_phashes:
                invalid_image_phashes.persist_remove(phash_x)
                print('-Invalid X')
            if phash_y in invalid_image_phashes:
                invalid_image_phashes.persist_remove(phash_y)
                print('-Invalid Y')
            print('Reset')

    yes_btn.on_click(on_yes)
    no_btn.on_click(on_no)
    related_btn.on_click(on_related)
    invalid_x_btn.on_click(on_invalid_x)
    invalid_y_btn.on_click(on_invalid_y)
    reset_btn.on_click(on_reset)

    if default == 'no':
        on_no(None)

    return HBox([VBox([yes_btn, no_btn, related_btn, invalid_x_btn, invalid_y_btn, reset_btn, output]),
                 widgets.Image(value=open(image_x['file_path'], 'rb').read(), width=250, height=150),
                 widgets.Image(value=open(image_y['file_path'], 'rb').read(), width=250, height=150)])

In [25]:
def potential_duplicates (threshold):
    for i in range(distance.shape[0]):
        for j in range(i):
            if distance[i, j] <= threshold:
                phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
                if (phash_pair not in auto_duplicated_image_phash_pairs and
                    phash_pair not in duplicated_post_image_phash_pairs and
                    phash_pair not in duplicated_image_phash_pairs and
                    phash_pair not in not_duplicated_image_phash_pairs and
                    phash_pair not in related_album_image_phash_pairs and
                    phash_pair not in related_image_phash_pairs):
                    yield (i, j)

In [26]:
distance_threshold = 13

In [27]:
pdup = potential_duplicates(distance_threshold)

In [28]:
for i in range(10):
    try:
        next_pdup = next(pdup)
    except StopIteration:
        print('StopIteration')
        break

    idx_x, idx_y = next_pdup
    image_x = imageDedup[idx_x]
    image_y = imageDedup[idx_y]
    print(f"{idx_x} {idx_y} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
    display(make_dedup_box(idx_x, idx_y, default=None if distance[idx_x, idx_y] < 9 else 'no'))

StopIteration


## Images with high variability

In [29]:
# interested_phashes = set()

In [30]:
# def potential_duplicates_high (threshold):
#     for i in range(distance.shape[0]):
#         for j in range(i):
#             if distance[i, j] >= threshold:
#                 phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
#                 if (phash_pair in duplicated_image_phash_pairs):
#                     interested_phashes.add(imageMetas[i]['phash'])
#                     interested_phashes.add(imageMetas[j]['phash'])
#                     yield (i, j)

In [31]:
# pduph = potential_duplicates_high(10)

In [32]:
# for i in range(100):
#     try:
#         next_pdup = next(pduph)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
# #     display(make_dedup_box(idx_x, idx_y))

In [33]:
# def potential_duplicates_interested (threshold):
#     for i in range(distance.shape[0]):
#         for j in range(i):
#             if distance[i, j] <= threshold:
#                 phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
#                 if (imageMetas[i]['phash'] in interested_phashes or
#                     imageMetas[j]['phash'] in interested_phashes) and (
#                     phash_pair not in auto_duplicated_image_phash_pairs and
#                     phash_pair not in duplicated_post_image_phash_pairs and
#                     phash_pair not in duplicated_image_phash_pairs and
#                     phash_pair not in not_duplicated_image_phash_pairs and
#                     phash_pair not in related_album_image_phash_pairs and
#                     phash_pair not in related_image_phash_pairs):
#                     yield (i, j)

In [34]:
# pdupi = potential_duplicates_interested(18)

In [35]:
# for i in range(10):
#     try:
#         next_pdup = next(pdupi)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
#     display(make_dedup_box(idx_x, idx_y))

In [36]:
# def potential_duplicates_related ():
#     for i in range(distance.shape[0]):
#         for j in range(i):
#             if related_distance[i, j] == 0:
#                 phash_pair = frozenset([imageDedup[i]['phash'], imageDedup[j]['phash']])
#                 if (phash_pair not in related_album_image_phash_pairs):
#                     yield (i, j)

In [37]:
# pdupr = potential_duplicates_related()

In [38]:
# for i in range(10):
#     try:
#         next_pdup = next(pdupr)
#     except StopIteration:
#         print('StopIteration')
#         break

#     idx_x, idx_y = next_pdup
#     image_x = imageDedup[idx_x]
#     image_y = imageDedup[idx_y]
#     print(f"{idx_x} {idx_y} {pdistance[idx_x, idx_y]} {distance[idx_x, idx_y]} {image_x['phash']} {image_y['phash']} {image_x['width']} {image_y['width']} {image_x['image_id']} {image_y['image_id']}")
#     display(make_dedup_box(idx_x, idx_y))

# Consolidate

## Related images

In [39]:
related_images = [[imageDedup[idx]['image_id'] for idx in c]
                     for c in nx.components.connected_components(nx.Graph(related_distance <= 1))
                     if len(c) > 1]
len(related_images)

124

In [40]:
for ids in related_images:
    for i in ids:
        imageMeta = imageDedup[image_id_to_idx(i)]
        ri = [r for r in set(imageMeta.get('related_images', []) + ids) if r != i]
        imagededup.update_one({'image_id': i}, {'$set': {'related_images': ri}})

## Duplicated images

In [41]:
class ImageDedup ():
    _attrs = [
        'id',
        'post_id',
        'datetime',
        'url',
        'title',
        'content',
        'author',
        'removed',
        'ups',
        'num_comments',
        'external_link',
        'source',
        'source_platform',
        'source_url',
        'tags',
        'labels',
        'media_type',
        'thumbnail_url',
        'preview_url',
        'external_link_url',
        'archive_url',

        'thumbnail',
        'preview',
        'external_link',
        'archive',
        'manual',

        'image_id',
        'short_image_id',
        'album',
        'index_in_album',
        'image_type',
        'file_path',
        'ext',
        'animated',
        'size',
        'width',
        'height',
        'pixels',
        'image_order',
        'ahash',
        'phash',
        'pshash',
        'dhash',
        'whash',

        'duplicated_posts',
        'related_images',
        'duplicated_images',
        'popularity_score'
    ]

    def __init__ (self, imageMetas=[]):
        if len(imageMetas) == 0:
            raise Exception('Empty imageFiles array.')

        self._imageMetas = imageMetas
        self._image_ids = [i['image_id'] for i in imageMetas]
        self._image_order = sort_images(self._imageMetas)

        self._post_ids = {i['post_id'] for i in imageMetas}
        self._posts = [posts.find_one({'post_id': i}) for i in self._post_ids]
        self._posts += [posts.find_one({'post_id': i}) for p in self._posts for i in p['duplicated_posts'] if 'duplicated_posts' in p]
        if None in self._posts:
            print(self._post_ids)
        self._post_order = sort_posts(self._posts)

        for k, v in self.main_image.items():
            if k in ['duplicated_posts', 'related_images']:
                continue
            setattr(self, k, v)

        for k, v in self.main_post.items():
            if k in ['duplicated_posts', 'related_images']:
                continue
            if k in ['preview', 'thumbnail', 'external_link', 'archive', 'manual']:
                setattr(self, f"{k}_url", v)
            else:
                setattr(self, k, v)

    def digest (self):
        return {a:getattr(self, a) for a in ImageDedup._attrs if hasattr(self, a)}

    @property
    def duplicated_posts (self):
        post_ids = self._post_ids.union(*[set(p.get('duplicated_posts', [])) for p in self._posts])
        return [i for i in post_ids if i != self.post_id]

    @property
    def duplicated_images (self):
        return [i for i in self._image_ids if i != self.image_id]

    @property
    def related_images (self):
        return [ri for i in self._imageMetas for ri in i.get('related_images', []) if ri != self.image_id]

    @property
    def main_post (self):
#         if len(self._post_order) > 1 and self._post_order[0]['source_platform'] != 'reddit':
#             print(f"main post warning: {[p['post_id'] for p in self._post_order]}")
        return self._post_order[0]

    @property
    def popularity_score (self):
        return sum([post_score(p) for p in self._posts])

    @property
    def main_image (self):
#         if len(self._image_order) > 1 and self._image_order[0]['source_platform'] != 'reddit':
#             print(f"main image warning: {[i['image_id'] for i in self._image_order]}")
        return self._image_order[0]

In [42]:
duplicated_images = [[imageDedup[idx]['image_id'] for idx in c]
                     for c in nx.components.connected_components(nx.Graph(distance <= 1))]

In [43]:
# imageDedup[image_id_to_idx('reddit/AusFinance/fman6b_0')]

In [44]:
def dedup_image (ids):
    imagedd = ImageDedup([imageDedup[image_id_to_idx(i)] for i in ids])
    if imagedd.main_post['source'] != 'dataisugly':
        print(f"Image not from dataisugly: {imagedd.main_post['post_id']}")
    for i in imagedd.duplicated_images:
        imagededup.delete_one({'image_id': i})
    imagededup.replace_one({'image_id': imagedd.image_id}, imagedd.digest(), upsert=True)
    return imagedd

In [45]:
imagedds = parallel(dedup_image, duplicated_images, n_jobs=1)

In [46]:
duplicated_image_ids = [c
                     for c in nx.components.connected_components(nx.Graph(distance <= 1))
                     if len(c) > 1]
# len(duplicated_image_ids)
cnt = 0
for idxs in duplicated_image_ids:
#     print(f"{[imageDedup[i]['image_id'] for i in idxs]}")
    if len(idxs) >= 4:
        display(HBox([
            widgets.Image(value=open(imageDedup[i]['file_path'], 'rb').read(), width=100, height=100)
            for i in idxs]))
        cnt += 1
        if cnt >= 20:
            break